In [1]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_text_splitters.character import RecursiveCharacterTextSplitter
from langgraph.prebuilt import create_react_agent
from langchain_core.tools import Tool
from dotenv import load_dotenv
from pypdf import PdfReader

In [2]:
file = open("C:\\Users\\Zafar\\Downloads\\AIML_Task_1[1].pdf","rb")
content = ""
# Create a pdf reader object
reader = PdfReader(file)

for page in reader.pages:
    # For each page in the pdf add it to the content string
    content += page.extract_text()

# Create a recursive text splitter object
splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n","\n","."],  # Characters on which to split the text
    chunk_size = 200, # Number of words in a chunk
    chunk_overlap = 50  # Numbers of words which will be overlapped with the previous chunk
)

chunks = splitter.split_text(content)

In [3]:
from pinecone import Pinecone, ServerlessSpec
from sentence_transformers import SentenceTransformer
import os
from dotenv import load_dotenv

# Load the environment variables
load_dotenv()

api_key = os.environ["PINECONE_API_KEY"]

# Initialize a pinecone client
client = Pinecone(api_key=api_key)  

# Create a dense index with seperate embeddings
index_name = "storage-py"   

# Load the model for generating sentence embeddings
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

In [4]:
if not client.has_index(index_name):
    client.create_index(
        name = index_name,
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1",
        ),
    )

In [5]:
records = []

embeddings = model.encode(chunks)

# Iterate over each chunk adding it to the vector database
for i in range(len(embeddings)):
    records.append({"id":"vec"+str(i+1),"values":embeddings[i].tolist(), "metadata":{"chunk_text":chunks[i]}})


d:\Codes\ai-engineer-roadmap-2025\.venv\lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [6]:
records[1]["metadata"]["chunk_text"]

'•Setting up and using theGemini API (Free Tier)for LLM access.\n•Building a modularAI Agentusing theLangChainframework.\n•Integrating tools that extend model capabilities:'

In [7]:
dense_index = client.Index(index_name)

# Upsert the records into a namespace
dense_index.upsert(namespace="example-namespace", vectors=records)

{'upserted_count': 31}

In [8]:
query = model.encode("What is the objective of the project?").tolist()

# Target the index
dense_index = client.Index(index_name)

results = dense_index.query(
namespace="example-namespace",
vector = query,
top_k=3,
include_values=True,
include_metadata=True
)

context = ""
# Add the text of each relevant chunk to the context
for hit in results["matches"]:
    context += hit["metadata"]["chunk_text"]

print(context)

d:\Codes\ai-engineer-roadmap-2025\.venv\lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [9]:
def query_vector_db(text):
    global model, client
    query = model.encode(text).tolist()

    # Target the index
    dense_index = client.Index(index_name)

    results = dense_index.query(
    namespace="example-namespace",
    vector = query,
    top_k=3,
    include_values=True,
    include_metadata=True
    )

    context = preprocess(results)
    return context

def preprocess(results):
    context = ""
    # Add the text of each relevant chunk to the context
    for hit in results["matches"]:
        context += hit["metadata"]["chunk_text"]

    return context

In [10]:
from langchain_tavily import TavilySearch

def query_tool(text):
    """
    RAG Tool to retrieve information from the indexed PDF in Pinecone.
    This tool does NOT access any local files. It searches within the
    PDF content that has already been embedded and stored.
    """

    # Carry out the user's query on the database
    context = query_vector_db(text)
    return context

# Create an llm object using Google class and gemini 2.5 flash model
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.7)


# Create a tool object using Tavily Search
search_tool = TavilySearch(
    max_results = 3,
    topic = "general",
    
)

rag_tool = Tool(
    name="rag",
    func=query_tool,
    description="Use this tool to answer questions **based on the PDF content** that has already been embedded and stored in Pinecone. The user may refer to it as 'the file'."
)

# Create an AI agent by merging the llm with the tool
agent = create_react_agent(llm, [search_tool, rag_tool])

In [12]:
resp = agent.invoke({"messages": "Hello"})

In [19]:
try:
    print(resp["messages"][3].content)
except IndexError:
    print(resp["messages"][1].content)

Hello! I'm a large language model, able to assist with a wide range of tasks and questions. What can I do for you today?
